## Generate and modify IDPP path 

In [1]:
from pymatgen_diffusion.neb.pathfinder import IDPPSolver
from pymatgen.core import Structure
import numpy as np
import os
from datetime import datetime

In [24]:
dirname = "/mnt/c/Liugy-wd/IDPP_test/pymatgen/pymatgen-diffusion/test/POSCAR_tri"

In [25]:
# run IDPP on linear interpolation
init_struct = Structure.from_file(os.path.join(dirname, "CONTCAR_00"), False)
final_struct = Structure.from_file(os.path.join(dirname, "CONTCAR_02"), False)
obj = IDPPSolver.from_endpoints(endpoints=[init_struct, final_struct], nimages=7, sort_tol=0, pi_bond=0.0)
# idpp_path = obj.run(maxiter=5000, tol=1e-5, gtol=1e-3, species=None)
# result = idpp_path

NotADirectoryError: [Errno 20] Not a directory: '/mnt/c/Liugy-wd/IDPP_test/pymatgen/pymatgen-diffusion/test/POSCAR_tri/CONTCAR_00'

In [ ]:
# run IDPP on given initial path
nimages = 5
images = []
for i in range(nimages):
    images.append(Structure.from_file(os.path.join(dirname,"CONTCAR-{:02d}".format(i)), False))
obj = IDPPSolver(images)
idpp_path = obj.run(maxiter=5000, tol=1e-5, gtol=1e-3, species=None)
result = idpp_path

In [13]:
# remove clash
import time
start_time = time.time()

method = 'decay'
clash_removed_path, attr_force_log, attr_index_log, rpl_force_log, rpl_index_log, disp_log = obj.clash_removal_NEB(obj.structures,step_size=0.05, max_iter=200,k_bonded=0.05,base_step=0.01, max_step=0.05, step_update_method=method)
# available_method = ('decay', 'expo', 'triangular')
result = clash_removed_path

print("--- %.02d seconds ---" % (time.time() - start_time))

10%
20%
30%
40%
50%
60%
70%
80%
90%
--- 249 seconds ---


In [17]:
# rerun
obj_rerun = IDPPSolver(clash_removed_path)
rerun_path = obj_rerun.rerun(maxiter = 5000, tol=1e-5, gtol=1e-3)
result = rerun_path

element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:C radius:0.7 ang
element:Li radius:0.83


## Saving data

In [16]:
# create saving folder
now = datetime.now()
today = now.strftime("%Y%m%d")
path = os.path.join(dirname, today)
if not os.path.exists(path):
    os.mkdir(path)
i = 1
saveDir = os.path.join(path,"%s_test_%02d"%(today,i))
while(os.path.exists(saveDir)): 
    i += 1
    saveDir = os.path.join(path,"%s_test_%02d"%(today,i))
os.mkdir(saveDir)


In [17]:
# save result structures as POSCAR
for n,struct in enumerate(idpp_path):
    # os.mkdir("%s\\%02d" % (saveDir,n))
    struct.to(fmt="poscar",filename="%s\\POSCAR_%02d" % (saveDir,n))

In [ ]:
# save f_log and disp_log
np.savetxt(os.path.join(saveDir, "attr_force_log.txt"), attr_force_log)
np.savetxt(os.path.join(saveDir, "rpl_force_log.txt"), rpl_force_log)
np.savetxt(os.path.join(saveDir, "attr_index_log.txt"), attr_index_log, fmt='%d')
np.savetxt(os.path.join(saveDir, "rpl_index_log.txt"), rpl_index_log, fmt='%d')
np.savetxt(os.path.join(saveDir, "disp_log.txt"), disp_log)

In [ ]:
# output log
with open(os.path.join(saveDir, "log.txt"),"a" ) as f:
    log = "r = %d \nmax_bond_length = %f\nradius = %f\nmax_iteration = %d\ntolerance = %d\ngtolerance%d" % (5, 4.09, 1.44, 5000, 1e-5, 1e-3)
    f.write(log)

# save probes
with open(os.path.join(saveDir, "d_3-4.txt"),"w") as f:
    for i in probe1:
        f.write("%f\n" % i)
with open(os.path.join(saveDir, "d_3-9.txt"),"w") as f:
    for i in probe2:
        f.write("%f\n" % i)
with open(os.path.join(saveDir, "d_3-10.txt"),"w") as f:
    for i in probe3:
        f.write("%f\n" % i)

np.savetxt(os.path.join(saveDir, "d_26.txt"),d_26)

with open(os.path.join(saveDir, "neighbor_26.txt"),"w") as f:
    for i in range(len(n_26)):
        f.write("iteration %d \n" % i)
        for ni in n_26[i]:
            f.write(str(ni)+"\n")

## Plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

path = saveDir
# path = "C:\\ComputMatSci\\IDPP_test\\pymatgen\\Ag_2\\20191029\\20191029_test_04"
attr_force = np.loadtxt("%s\\attr_force_log.txt" % path)
rpl_force = np.loadtxt("%s\\rpl_force_log.txt" % path)
disp = np.loadtxt("%s\\disp_log.txt" % path)
num_of_images = len(attr_force[0])
y_lb = 0
y_ub = 0.5

plt.figure(figsize=[6.4, 9])

ax1 = plt.subplot(311)
for i in range(num_of_images):
    plt.plot(attr_force[:,i],label='image %02d' % i)
ax1.legend(loc="upper right",fontsize='small')
ax1.set_title("attractive force")

plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = plt.subplot(312)
for i in range(num_of_images):
    plt.plot(rpl_force[:,i],label='image %02d' % i)
ax2.legend(loc="upper right",fontsize='small')
ax2.set_ylim(0,0.5)
ax2.set_title("repulsive force")
plt.setp(ax2.get_xticklabels(),visible=False)

ax3 = plt.subplot(313)
for i in range(num_of_images):
    plt.plot(disp[:,i],label='image %02d' % i)
ax3.legend(loc="upper right",fontsize='small')
ax3.set_title("displacement")

# plt.title("different step size dynamics (all initial step sizes are 0.005 ")
plt.tight_layout()
plt.savefig("%s\\%s.png"%(saveDir,method),dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

path = "C:\\ComputMatSci\\IDPP_test\\pymatgen\\Ag_2\\20191028"
log = "attr_force_log.txt"
attr_force_03 = np.loadtxt("%s\\20191028_test_01\\%s" % (path, log))
attr_force_04 = np.loadtxt("%s\\20191028_test_04\\%s" % (path, log))
attr_force_05 = np.loadtxt("%s\\20191028_test_05\\%s" % (path, log))
attr_force_06 = np.loadtxt("%s\\20191028_test_06\\%s" % (path, log))
attr_force_01 = np.loadtxt("C:\\ComputMatSci\\IDPP_test\\pymatgen\\Ag_2\\20191029\\20191029_test_02\\%s" % log)

x = np.arange(200)
y_lb = 0
y_ub = 0.5
plt.figure(figsize=[6.4, 15])

ax1 = plt.subplot(511)
plt.plot(x, attr_force_03)
ax1.set_title("decay 0.01")
ax1.set_ylim(y_lb, y_ub)
plt.setp(ax1.get_xticklabels(),visible=False)

ax2 = plt.subplot(512)
plt.plot(attr_force_04)
ax2.set_title("linear increase")
ax2.set_ylim(y_lb, y_ub)
plt.setp(ax2.get_xticklabels(),visible=False)

ax3 = plt.subplot(513)
plt.plot(attr_force_05)
ax3.set_title("exponentially increase *2")
ax3.set_ylim(y_lb, y_ub)
plt.setp(ax3.get_xticklabels(),visible=False)

ax4 = plt.subplot(514)
plt.plot(attr_force_06)
ax4.set_title("*1.2 or reject")
ax4.set_ylim(y_lb, y_ub)

ax5 = plt.subplot(515)
plt.plot(attr_force_01)
ax5.set_title("cyclic step size")
ax5.set_ylim(y_lb, y_ub)

# plt.title("different step size dynamics (all initial step sizes are 0.005 ")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def _get_triangular_step(iteration, half_period=5, base_step=0.05, max_step=0.1):
    """
    Given the inputs, calculates the step_size that should be applicable
    for this iteration using CLR technique from Anand Saha
    (http://teleported.in/posts/cyclic-learning-rate/).
    """
    cycle = np.floor(1 + iteration/(2 * half_period))
    x = np.abs(iteration/half_period - 2 * cycle + 1)
    step = base_step + (max_step - base_step) * np.maximum(0, (1-x))
    return step

x = np.arange(200)
y = np.array(list(map(_get_triangular_step, x)))
plt.plot(x,y)
plt.show()

## Testing

In [ ]:
# check d_26
count = 0
for i in d_26:
    for j in i:
        diff = abs(j - 2.88)
        if (diff < 0.01 ):
            count += 1
print(count)

In [51]:
from pymatgen.io.lammps.data import lattice_2_lmpbox as box
struct = Structure.from_file("/mnt/c/Liugy-wd/IDPP_test/pymatgen/pymatgen-diffusion/test/CONTCAR")
lmpbox, sym = box(struct.lattice)
struct.to(fmt="lammps",filename="/mnt/c/Liugy-wd/IDPP_test/pymatgen/pymatgen-diffusion/test/dump-CONTCAR")

0.000000 23.112000  xlo xhi
0.000000 20.015579  ylo yhi
0.000000 17.076599  zlo zhi
-11.556000 0.000000 0.000000  xy xz yz

In [19]:
b_frac = [init_struct[i].frac_coords for i in range(len(init_struct))]
init_struct

Structure Summary
Lattice
    abc : 12.133600235 12.133600235 12.133600235
 angles : 90.0 90.0 90.0
 volume : 1786.3602509741534
      A : 12.133600235 0.0 0.0
      B : 0.0 12.133600235 0.0
      C : 0.0 0.0 12.133600235
PeriodicSite: Al (0.0072, 12.1252, 12.1198) [0.0006, 0.9993, 0.9989]
PeriodicSite: Al (12.1243, 2.0220, 2.0211) [0.9992, 0.1666, 0.1666]
PeriodicSite: Al (2.0260, 0.0026, 2.0215) [0.1670, 0.0002, 0.1666]
PeriodicSite: Al (2.0340, 2.0220, 12.1206) [0.1676, 0.1666, 0.9989]
PeriodicSite: Al (4.0472, 12.1169, 12.1245) [0.3336, 0.9986, 0.9992]
PeriodicSite: Al (4.0475, 2.0220, 2.0129) [0.3336, 0.1666, 0.1659]
PeriodicSite: Al (6.0686, 12.1259, 2.0152) [0.5001, 0.9994, 0.1661]
PeriodicSite: Al (6.0704, 2.0220, 12.1323) [0.5003, 0.1666, 0.9999]
PeriodicSite: Al (8.0863, 12.1307, 12.1303) [0.6664, 0.9998, 0.9997]
PeriodicSite: Al (8.0879, 2.0220, 2.0167) [0.6666, 0.1666, 0.1662]
PeriodicSite: Al (10.1145, 0.0064, 2.0248) [0.8336, 0.0005, 0.1669]
PeriodicSite: Al (10.1063, 2.0

In [23]:
init_struct.lattice

Lattice
    abc : 12.133600235 12.133600235 12.133600235
 angles : 90.0 90.0 90.0
 volume : 1786.3602509741534
      A : 12.133600235 0.0 0.0
      B : 0.0 12.133600235 0.0
      C : 0.0 0.0 12.133600235